In [1]:
import numpy as np
import pandas as pd

In [2]:
from itertools import permutations

In [3]:
tcga_meta = pd.read_csv('../data/processed/toil_tcga_sample_info_140722.csv')

In [4]:
annot = pd.read_csv('../data/processed/ensembl_annotation_050722.csv', low_memory=False)

In [5]:
tcga_meta

,project,cancer_type,primary_site,sample,gender,sample_type,_sample_stem
0,UVM,uveal melanoma,eye,TCGA-V4-A9EE-01,male,primary tumor,TCGA-V4-A9EE
1,UVM,uveal melanoma,eye,TCGA-VD-AA8N-01,male,primary tumor,TCGA-VD-AA8N
2,UVM,uveal melanoma,eye,TCGA-V4-A9EI-01,male,primary tumor,TCGA-V4-A9EI
3,UVM,uveal melanoma,eye,TCGA-VD-AA8O-01,male,primary tumor,TCGA-VD-AA8O
4,UVM,uveal melanoma,eye,TCGA-WC-A888-01,male,primary tumor,TCGA-WC-A888
...,...,...,...,...,...,...,...
9870,ACC,adrenocortical cancer,adrenal gland,TCGA-OR-A5J3-01,female,primary tumor,TCGA-OR-A5J3
9871,ACC,adrenocortical cancer,adrenal gland,TCGA-OR-A5KY-01,female,primary tumor,TCGA-OR-A5KY
9872,ACC,adrenocortical cancer,adrenal gland,TCGA-OR-A5J9-01,female,primary tumor,TCGA-OR-A5J9
9873,ACC,adrenocortical cancer,adrenal gland,TCGA-OR-A5JE-01,female,primary tumor,TCGA-OR-A5JE


In [6]:
annot.head()

,chromosome_or_scaffold,ensembl_gene_id,ensembl_gene_name,gene_type,strand,gene_start,gene_end,ensembl_trs_id,ensembl_trs_name,trs_type,...,cds_end,cds_length_bp,in_gencode_basic,appris_annotation,ensembl_protein_id,uniprot_base_id,uniprot_isoform_id,uniprot_trembl_id,ccds_id,protein_length_aa
0,MT,ENSG00000210049,MT-TF,Mt_tRNA,1,577,647,ENST00000387314,MT-TF-201,Mt_tRNA,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MT,ENSG00000211459,MT-RNR1,Mt_rRNA,1,648,1601,ENST00000389680,MT-RNR1-201,Mt_rRNA,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MT,ENSG00000210077,MT-TV,Mt_tRNA,1,1602,1670,ENST00000387342,MT-TV-201,Mt_tRNA,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MT,ENSG00000210082,MT-RNR2,Mt_rRNA,1,1671,3229,ENST00000387347,MT-RNR2-201,Mt_rRNA,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MT,ENSG00000209082,MT-TL1,Mt_tRNA,1,3230,3304,ENST00000386347,MT-TL1-201,Mt_tRNA,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
coad_data = pd.read_csv('../data/processed/toil/toil_tcga_coad.csv')

In [10]:
coad_data = coad_data.rename({'sample': 'ensembl_trs_id'}, axis=1)
#coad_data['ensembl_trs_id'] = coad_data['ensembl_trs_id'].map(lambda x: x[:x.index('.')])
coad_data['ensembl_gene_id'] = coad_data['ensembl_trs_id'].map(annot.set_index('ensembl_trs_id')['ensembl_gene_id'])

In [11]:
def prefilter(df):
    df_gene_median_tpm = df.drop('ensembl_trs_id', axis=1).groupby('ensembl_gene_id').sum().replace(0.0, np.nan).median(0, skipna=True).mean()
    df_trs_median_tpm = df.drop('ensembl_gene_id', axis=1).set_index('ensembl_trs_id').replace(0.0, np.nan).median(0, skipna=True).mean()
    
    trs_tpm_sum = df.drop('ensembl_gene_id', axis=1).set_index('ensembl_trs_id').max(1)
    
    df = df[df['ensembl_trs_id'].isin(trs_tpm_sum[trs_tpm_sum > df_trs_median_tpm].index)]
    
    genes_n_trs = df.groupby('ensembl_gene_id').size()
    genes_tpm_sum = df.drop('ensembl_trs_id', axis=1).groupby('ensembl_gene_id').sum().max(1)
    
    genes_ok = (genes_n_trs[genes_n_trs > 1].index.intersection(genes_tpm_sum[genes_tpm_sum > df_gene_median_tpm].index))
    
    return df[df['ensembl_gene_id'].isin(genes_ok)]

In [13]:
coad_data_f = prefilter(coad_data)

In [14]:
coad_data_f = coad_data_f.set_index(['ensembl_gene_id', 'ensembl_trs_id'])

In [15]:
coad_data_f = coad_data_f.sort_index()

In [16]:
def tcga_group(ctype, iscancer=True):
    if iscancer:
        return tcga_meta[(tcga_meta['project'] == ctype) & (tcga_meta['sample_type'] == 'primary tumor')]['sample'].to_list()
    else:
        return tcga_meta[(tcga_meta['project'] == ctype) & (tcga_meta['sample_type'] == 'normal tissue')]['sample'].to_list()

In [34]:
def mdt_2x_approach(df):
    res = pd.DataFrame(index=df.index.get_level_values(0).unique(), columns=df.columns)
    
    for gene, part in df.groupby(level=0):
        part = part.droplevel(0)
            
        cond = part.divide(np.sort(part, axis=0)[::-1][1]).max(0)
        res.loc[gene] = part.idxmax().where(cond > 2, np.nan).values
    
    return res

def mdt_m1_approach(df):
    res = pd.DataFrame(index=df.index.get_level_values(0).unique(), columns=df.columns)
    
    for gene, part in df.groupby(level=0):
        part = part.droplevel(0)
            
        cond = part.subtract(np.sort(part, axis=0)[::-1][1]).max(0)
        res.loc[gene] = part.idxmax().where(cond > 1, np.nan).values
    
    return res

def mdt_approach(df, min_diff = 0):
    res = pd.DataFrame(index=df.index.get_level_values(0).unique(), columns=df.columns)
    
    for gene, part in df.groupby(level=0):
        part = part.droplevel(0)
            
        cond = part.subtract(np.sort(part, axis=0)[::-1][1]).max(0)
        res.loc[gene] = part.idxmax().where(cond > min_diff, np.nan).values
    
    return res

In [28]:
def pair_freq_approach(df, scanc=None, snorm=None):
    
    res = pd.DataFrame()
    
    for gene, part in df.groupby(level=0):
        part = part.droplevel(0)
        
        for i, j in permutations(part.index, 2):
            print(part.index)
            break
        break

In [26]:
pair_freq_approach(coad_data_f, )

[(1, 2), (2, 1)]

In [30]:
coad_norm = coad_data_f[tcga_group('COAD', False)]

In [35]:
coad_norm_mdt = mdt_approach(coad_norm)

In [31]:
coad_cancer = coad_data_f[tcga_group('COAD')]

In [36]:
%%time
coad_cancer_mdt = mdt_approach(coad_cancer)

CPU times: total: 40.9 s
Wall time: 41.2 s


In [27]:
def extract_dominant_form(isoform_list):
    isoform_counts = isoform_list.value_counts(normalize=True, dropna=False)
    if not isoform_counts.index.isnull()[0]:
        if isoform_counts[0] >= 0.75:
            return isoform_counts.index[0]
        else:
            return np.nan
    else:
        return np.nan

In [29]:
def extract_dominant_form_freq(isoform_list):
    isoform_counts = isoform_list.value_counts(normalize=True, dropna=False)
    if not isoform_counts.index.isnull()[0]:
        if isoform_counts[0] >= 0.75:
            return isoform_counts[0]
        else:
            return np.nan
    else:
        return np.nan

In [48]:
%%time
coad_norm_cons = coad_norm_mdt.apply(extract_dominant_form, axis=1)
coad_cancer_cons = coad_cancer_mdt.apply(extract_dominant_form, axis=1)

coad_norm_cons_freq = coad_norm_mdt.apply(extract_dominant_form_freq, axis=1)
coad_cancer_cons_freq = coad_cancer_mdt.apply(extract_dominant_form_freq, axis=1)

CPU times: total: 35.9 s
Wall time: 36.3 s


In [53]:
coad_cons = (coad_cancer_cons != coad_norm_cons)

In [40]:
coad_cons = (coad_norm_cons != coad_cancer_cons)

In [54]:
coad_cons

ensembl_gene_id
ENSG00000000003    False
ENSG00000000005    False
ENSG00000000419    False
ENSG00000000457     True
ENSG00000000460     True
                   ...  
ENSG00000288573    False
ENSG00000288596     True
ENSG00000288920    False
ENSG00000289565     True
ENSG00000289685    False
Length: 16431, dtype: bool

In [42]:
coad_cons = coad_cons[coad_cons]

In [55]:
coad

ensembl_gene_id
ENSG00000000003    ENST00000373020
ENSG00000000005    ENST00000373031
ENSG00000000419    ENST00000371588
ENSG00000000457    ENST00000367772
ENSG00000000460                NaN
                        ...       
ENSG00000288573    ENST00000446167
ENSG00000288596                NaN
ENSG00000288920    ENST00000484897
ENSG00000289565                NaN
ENSG00000289685    ENST00000272418
Length: 16431, dtype: object

In [56]:
coad_cancer_cons

ensembl_gene_id
ENSG00000000003    ENST00000373020
ENSG00000000005    ENST00000373031
ENSG00000000419    ENST00000371588
ENSG00000000457                NaN
ENSG00000000460    ENST00000413811
                        ...       
ENSG00000288573    ENST00000446167
ENSG00000288596                NaN
ENSG00000288920    ENST00000484897
ENSG00000289565                NaN
ENSG00000289685    ENST00000272418
Length: 16431, dtype: object

In [151]:
coad_cons = coad_cons.to_frame()

In [152]:
coad_cons['isoform_norm'] = coad_cons.index.map(coad_norm_cons)
coad_cons['isoform_cancer'] = coad_cons.index.map(coad_cancer_cons)

In [153]:
coad_cons.drop(0, axis=1).dropna()

,isoform_norm,isoform_cancer
ensembl_gene_id,,
ENSG00000089177,ENST00000408042,ENST00000354981
ENSG00000104635,ENST00000359741,ENST00000381237
ENSG00000113648,ENST00000312469,ENST00000304332
ENSG00000129009,ENST00000249842,ENST00000395118
ENSG00000163520,ENST00000404922,ENST00000295760
ENSG00000167748,ENST00000301420,ENST00000596300
ENSG00000181885,ENST00000397317,ENST00000360325
ENSG00000197249,ENST00000404814,ENST00000393087


In [159]:
coad_norm.loc['ENSG00000104635']

,TCGA-AA-3525-11,TCGA-A6-2671-11,TCGA-A6-5659-11,TCGA-A6-2675-11,TCGA-AA-3697-11,TCGA-AZ-6605-11,TCGA-AA-3489-11,TCGA-AZ-6599-11,TCGA-A6-2682-11,TCGA-AA-3655-11,...,TCGA-A6-2684-11,TCGA-AA-3531-11,TCGA-AA-3516-11,TCGA-A6-2679-11,TCGA-A6-2680-11,TCGA-A6-2683-11,TCGA-AA-3660-11,TCGA-A6-2685-11,TCGA-A6-2678-11,TCGA-A6-5667-11
ensembl_trs_id,,,,,,,,,,,,,,,,,,,,,
ENST00000289952,1.92994,4.22004,0.67001,2.54001,1.37999,0.49000,1.66006,3.38012,3.46003,2.77999,...,4.70011,4.64987,3.21998,1.49002,1.97996,2.44005,2.10003,3.86007,2.76999,7.62010
ENST00000359741,25.57999,38.44869,30.36953,18.24995,22.07030,3.19995,10.31992,29.88919,29.33093,21.18002,...,33.65986,56.71957,36.48067,22.82965,49.73091,36.45034,28.59021,48.72095,31.21037,44.54117
ENST00000381237,10.00019,8.93978,8.30009,18.23984,18.48037,12.94020,12.10038,17.93020,13.77992,11.68979,...,10.52000,8.12982,7.20001,7.64020,28.94917,15.47006,11.88013,10.66983,13.26995,10.38019
ENST00000517370,0.43001,0.27001,0.26000,0.22000,0.20000,0.20000,0.18999,1.08003,0.33999,0.00000,...,0.70999,0.38001,0.27999,0.23001,0.82999,0.67001,0.49000,0.22000,0.50001,1.58996
ENST00000518348,0.20000,0.09000,0.12000,0.18000,0.09000,0.00000,0.00000,0.29000,0.12000,0.07000,...,0.12000,0.15000,0.00000,0.00000,0.24000,0.11000,0.15000,0.21000,0.31999,0.26000
ENST00000520644,0.00000,0.22000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.18000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
ENST00000520832,0.38001,1.09004,0.12000,0.64999,0.41000,0.23001,0.13000,1.11003,0.50001,0.49000,...,0.53001,0.49000,0.10000,0.64000,0.57001,0.53001,0.07000,0.21000,0.26000,0.60998
ENST00000524285,0.16000,1.17999,0.57001,0.33000,0.51000,0.00000,0.20000,1.12001,0.82003,0.64000,...,0.70999,0.80001,0.11000,0.54999,1.01001,0.48000,0.00000,0.54999,0.82003,2.06996


In [160]:
coad_cancer.loc['ENSG00000104635']

,TCGA-G4-6298-01,TCGA-G4-6626-01,TCGA-AA-3675-01,TCGA-AD-6895-01,TCGA-AD-6899-01,TCGA-AZ-4615-01,TCGA-G4-6627-01,TCGA-D5-5537-01,TCGA-A6-5659-01,TCGA-A6-6653-01,...,TCGA-4T-AA8H-01,TCGA-CA-5254-01,TCGA-F4-6461-01,TCGA-5M-AAT6-01,TCGA-A6-5667-01,TCGA-G4-6309-01,TCGA-CM-5344-01,TCGA-G4-6307-01,TCGA-CM-6161-01,TCGA-A6-A565-01
ensembl_trs_id,,,,,,,,,,,,,,,,,,,,,
ENST00000289952,0.00000,0.10000,0.31999,0.07000,1.53003,0.33999,0.27001,0.15000,0.08000,0.38999,...,1.12001,0.33000,0.51000,0.20000,0.65999,0.00000,0.00000,0.02000,0.86001,0.80001
ENST00000359741,1.57996,1.19002,1.69003,5.98006,1.11003,23.87021,11.27982,3.27992,0.52334,7.51987,...,8.18978,54.29974,3.11005,1.29001,0.70001,4.15991,1.96001,1.64995,3.88989,21.11991
ENST00000381237,12.10038,19.14937,38.82901,31.70096,24.03958,85.46034,41.42889,40.50040,9.13989,29.30045,...,33.84938,36.42003,23.73986,22.99961,43.08956,43.90055,13.64021,24.26055,62.59072,20.30030
ENST00000517370,0.00000,0.00000,0.20000,0.40001,0.10000,0.52002,0.47000,0.38001,0.02333,0.00000,...,1.38998,1.37999,0.29000,0.10000,0.00000,0.31999,0.26000,0.05000,0.27001,1.05998
ENST00000518348,0.03000,0.00000,0.08000,0.29000,0.13000,0.29999,0.18999,0.21000,0.06333,0.00000,...,0.81001,0.64999,0.16000,0.04000,0.08000,0.23001,0.12000,0.02000,0.27999,0.42000
ENST00000520644,0.00000,0.07000,0.00000,0.00000,0.18000,0.00000,0.00000,0.00000,0.02667,0.00000,...,0.27001,0.00000,0.25001,0.00000,0.00000,0.22000,0.00000,0.00000,0.00000,0.00000
ENST00000520832,0.06000,0.18999,0.14000,0.07000,0.11000,1.01001,0.64999,0.21000,0.08667,0.31000,...,0.17000,0.43001,0.15000,0.17000,0.34999,0.33000,0.31999,0.10000,0.20000,0.16000
ENST00000524285,0.00000,0.00000,0.37000,0.17000,0.09000,1.49998,1.09004,0.45999,0.08333,0.21000,...,0.00000,1.08003,0.00000,0.00000,0.16000,0.37000,0.14000,0.00000,0.23001,0.31000


In [57]:
tcga_groups = pd.read_csv('../data/processed/toil_tcga_group_size_140722.csv')

In [58]:
tcga_groups = tcga_groups[tcga_groups['normal'] >= 9]['project'].str.lower().to_list()

In [61]:
cons_cancer = pd.DataFrame(index=annot['ensembl_gene_id'].unique())
cons_norm = pd.DataFrame(index=annot['ensembl_gene_id'].unique())


cons_cancer_freq = pd.DataFrame(index=annot['ensembl_gene_id'].unique())
cons_norm_freq = pd.DataFrame(index=annot['ensembl_gene_id'].unique())

for i, g in enumerate(tcga_groups[:3]):
    tcga_part = pd.read_csv(f'../data/processed/toil/toil_tcga_{g}.csv')
    
    tcga_part = tcga_part.rename({'sample': 'ensembl_trs_id'}, axis=1)
    tcga_part['ensembl_gene_id'] = tcga_part['ensembl_trs_id'].map(annot.set_index('ensembl_trs_id')['ensembl_gene_id'])
    
    tcga_part_f = prefilter(tcga_part)
    
    tcga_part_f = tcga_part_f.set_index(['ensembl_gene_id', 'ensembl_trs_id'])
    tcga_part_f = tcga_part_f.sort_index()
    
    tcga_part_norm_mdt = mdt_approach(tcga_part_f[tcga_group(g.upper(), False)])
    tcga_part_cancer_mdt = mdt_approach(tcga_part_f[tcga_group(g.upper())])
    
    tcga_part_norm_cons = tcga_part_norm_mdt.apply(extract_dominant_form, axis=1)
    tcga_part_cancer_cons = tcga_part_cancer_mdt.apply(extract_dominant_form, axis=1)
    
    tcga_part_norm_cons_freq = tcga_part_norm_mdt.apply(extract_dominant_form_freq, axis=1)
    tcga_part_cancer_cons_freq = tcga_part_cancer_mdt.apply(extract_dominant_form_freq, axis=1)
    
    cons_norm[g] = cons_norm.index.map(tcga_part_norm_cons)
    cons_cancer[g] = cons_cancer.index.map(tcga_part_cancer_cons)
    
    cons_norm_freq[g] = cons_norm.index.map(tcga_part_norm_cons_freq)
    cons_cancer_freq[g] = cons_cancer.index.map(tcga_part_cancer_cons_freq)
    
    print(f'{i+1} / {len(tcga_groups)}   ', end='\r')

In [174]:
cons_norm = cons_norm[~cons_norm.index.duplicated()]
cons_cancer = cons_cancer[~cons_cancer.index.duplicated()]

In [176]:
diff = (cons_cancer != cons_norm).stack()

diff = diff[diff].to_frame()

diff['isoform_cancer'] = diff.index.map(cons_cancer.stack())
diff['isoform_norm'] = diff.index.map(cons_norm.stack())

diff = diff.drop(0, axis=1).dropna()

In [183]:
diff = diff.rename({'level_1': 'cancer_type'}, axis=1)

In [184]:
diff['cancer_type'] = diff['cancer_type'].str.upper()

In [190]:
diff.groupby('ensembl_gene_id').size().sort_values(ascending=False)[:10]

ensembl_gene_id
ENSG00000163520    6
ENSG00000164576    5
ENSG00000101290    5
ENSG00000113648    4
ENSG00000152492    4
ENSG00000129009    4
ENSG00000243147    3
ENSG00000158887    3
ENSG00000106404    3
ENSG00000182871    3
dtype: int64

In [192]:
diff = diff.set_index(['ensembl_gene_id', 'cancer_type'])

In [194]:
diff.to_excel('../reports/cons_mdt_results_170922.xlsx')

In [22]:
list(permutations([1,2,3], 2))

[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)]

In [ ]:
def calc_freq(df, cforms, nforms):
    